In [17]:
# Import required libraries and dependencies

# https://stackoverflow.com/questions/69596239/how-to-avoid-memory-leak-when-dealing-with-kmeans-for-example-in-this-code-i-am
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import hvplot.pandas
import holoviews as hv
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

## Function to run KMeans and Elbow chart to optimize code

In [5]:
# Function to call Kmeans on dataframe with variable clusters

def Kmeans (dataframe, num_clusters):
    
    # Initialize the K-Means model using the best value for k
    model = KMeans(n_clusters=num_clusters, random_state=1)
    
    # Fit the K-Means model 
    model.fit(dataframe)
    
    # Predict the clusters 
    clusters = model.predict(dataframe)
    
    return clusters

In [6]:
def Elbow (dataframe):
    
    # Create a list with the number of k-values from 1 to 11
    k = list(range(1, 11))
    
    # Create an empty list to store the inertia values
    inertia = []


    # Create a for loop to compute the inertia with each possible value of k
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    # 2. Fit the model to the data using `df_market_data_scaled`
    # 3. Append the model.inertia_ to the inertia list

    for i in k:
        k_model = KMeans(n_clusters=i, random_state=1)
        k_model.fit(dataframe)
        inertia.append(k_model.inertia_)

    # Create a dictionary with the data to plot the Elbow curve
    elbow_data = {"k": k, "inertia": inertia}
    
    return pd.DataFrame(elbow_data)

### Prepare the Data

In [7]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

df_market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

df_market_data_scaled[0:5]



array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [9]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original data
df_market_data_scaled = pd.DataFrame(df_market_data_scaled, columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])

# Set the coinid column as index
df_market_data_scaled['coin_id'] = df_market_data.index
df_market_data_scaled.set_index('coin_id', inplace=True)

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [10]:
# # Create a list with the number of k-values from 1 to 11
# k = list(range(1, 11))


In [11]:
# # Create an empty list to store the inertia values
# inertia = []


# # Create a for loop to compute the inertia with each possible value of k
# # Inside the loop:
# # 1. Create a KMeans model using the loop counter for the n_clusters
# # 2. Fit the model to the data using `df_market_data_scaled`
# # 3. Append the model.inertia_ to the inertia list

# for i in k:
#     k_model = KMeans(n_clusters=i, random_state=1)
#     k_model.fit(df_market_data_scaled)
#     inertia.append(k_model.inertia_)

In [18]:
# # Create a dictionary with the data to plot the Elbow curve
# elbow_data = {"k": k, "inertia": inertia}

# # Create a DataFrame with the data to plot the Elbow curve
# df_elbow_original = pd.DataFrame(elbow_data)

# Elbow function replaces above cells for optimization
df_elbow_original = Elbow(df_market_data_scaled)
df_elbow_original.head()

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.405923


In [22]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
original_plot = df_elbow_original.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks='k',
    label='Original Data'
)

original_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Arguably, the best cluster value is 4. 

## Double-Check the clsuter number by running Calinski Harabasz loops using Kmeans, Agglo and Birch.

In [23]:

# Declare empty arrays to store the Calinski Harabasz scores for the three models.
score_kmeans = []
score_agglomerative = []
score_birch = []


#Make a list of cluster sizes to test
k = list(range(2, 11))

# Test each cluster size for its CH score using each of the three models.
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(df_market_data_scaled)
    labels = k_model.labels_
    score = metrics.calinski_harabasz_score(df_market_data_scaled, labels)    
    score_kmeans.append({i:score})
    
    agglo_model = AgglomerativeClustering(n_clusters=i)
    agglo_predictions = agglo_model.fit_predict(df_market_data_scaled)
    labels = agglo_model.labels_
    score = metrics.calinski_harabasz_score(df_market_data_scaled, labels)    
    score_agglomerative.append({i:score})    
    
    birch_model = Birch(n_clusters=i)
    birch_model.fit(df_market_data_scaled)
    labels = birch_model.labels_
    score = metrics.calinski_harabasz_score(df_market_data_scaled, labels)    
    score_birch.append({i:score})

In [24]:
# Show the CH scores to confirm four clusters with KMeans is the best result. (4: 32.45985294262567)

print('KMeans:' + str(score_kmeans))
print('-'*7)
print('Agglo:' + str(score_agglomerative))
print('-'*7)
print('Birch:' + str(score_birch))

KMeans:[{2: 18.159572714246874}, {3: 25.264783436390285}, {4: 32.45985294262567}, {5: 30.554454737370296}, {6: 30.985560337533226}, {7: 31.367257898388512}, {8: 31.349334199067002}, {9: 31.33912541792401}, {10: 31.58227429825022}]
-------
Agglo:[{2: 18.399507179694112}, {3: 25.264783436390285}, {4: 28.413891756388992}, {5: 30.023823023350158}, {6: 29.858770473694246}, {7: 30.467051842174403}, {8: 31.166775607219474}, {9: 30.667150777463192}, {10: 31.005396203218183}]
-------
Birch:[{2: 18.399507179694112}, {3: 25.264783436390285}, {4: 28.413891756388992}, {5: 27.695262932216924}, {6: 29.126883731877893}, {7: 29.15903640200133}, {8: 28.71761645596914}, {9: 27.594527704898685}, {10: 27.52923366208563}]


---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [25]:
# Initialize the K-Means model using the best value for k
# model = KMeans(n_clusters=4, random_state=1)

In [26]:
# Fit the K-Means model using the scaled data
# model.fit(df_market_data_scaled)


In [27]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# original_clusters = model.predict(df_market_data_scaled)


# # Print the resulting array of cluster values.
# print(original_clusters)

In [28]:
# Call Function for Kmeans to replace the cells above

original_clusters = Kmeans(df_market_data_scaled, 4)

# Print the resulting array of cluster values.
print(original_clusters)

[3 3 1 1 3 3 3 3 3 1 1 1 1 3 1 3 1 1 3 1 1 3 1 1 1 1 1 1 3 1 1 1 0 3 1 1 2
 1 1 1 1]


In [29]:
# Create a copy of the DataFrame
df_market_data_predictions = df_market_data_scaled.copy()

In [30]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_predictions['clusters'] = original_clusters

# Display sample data
df_market_data_predictions

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,3
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,3
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,3
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,3


In [31]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# https://www.w3schools.com/colors/colors_groups.asp


original_scatter = df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols="coin_id",
    label='clusters',
    title='Original Scatter',
    color=['Gold', 'Orange', 'Crimson', 'DarkRed']
).opts(yformatter="%.0f")

original_scatter

:NdOverlay   [clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [32]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [33]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_scaled_pca = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_data_scaled_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [34]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

round(sum(pca.explained_variance_ratio_)*100, 2)


89.5

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

In [35]:
# Create a new DataFrame with the PCA data.
df_market_data_scaled_pca = pd.DataFrame(
    market_data_scaled_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)


#Copy the crypto names from the original data
df_market_data_scaled_pca['coin_id'] = df_market_data_predictions.index

# Set the coinid column as index
df_market_data_scaled_pca.set_index('coin_id', inplace=True)

# Display sample data
df_market_data_scaled_pca.head()


,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [36]:
# # Create a list with the number of k-values from 1 to 11
# k = list(range(1, 11))

In [37]:
# # Create an empty list to store the inertia values
# inertia = []

# # Create a for loop to compute the inertia with each possible value of k
# # Inside the loop:
# # 1. Create a KMeans model using the loop counter for the n_clusters
# # 2. Fit the model to the data using `df_market_data_pca`
# # 3. Append the model.inertia_ to the inertia list


# for i in k:
#     k_model = KMeans(n_clusters=i, random_state=1)
#     k_model.fit(df_market_data_scaled_pca)
#     inertia.append(k_model.inertia_)

In [38]:
# # Create a dictionary with the data to plot the Elbow curve
# elbow_data = {"k": k, "inertia": inertia}


# # Create a DataFrame with the data to plot the Elbow curve
# df_elbow_pca = pd.DataFrame(elbow_data)

df_elbow_pca = Elbow(df_market_data_scaled_pca)
df_elbow_pca.head()

,k,inertia
0,1,256.874086
1,2,165.901994
2,3,93.774626
3,4,49.665497
4,5,38.352251


In [40]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

pca_plot = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks='k', 
    line_dash='dashed',
    label='PCA Data'
)

pca_plot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Still four.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [41]:
# Initialize the K-Means model using the best value for k
# model = KMeans(n_clusters=4, random_state=0)


In [42]:
# Fit the K-Means model using the PCA data
# model.fit(df_market_data_scaled_pca)


In [43]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# pca_clusters = model.predict(df_market_data_scaled_pca)

#Function replaces starter code in above cells
pca_clusters = Kmeans(df_market_data_scaled_pca, 4)

# Print the resulting array of cluster values.
print(pca_clusters)

# Add 4 to the clusters so they are distinct from original data clusters
pca_clusters = pca_clusters + 4

print(pca_clusters)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]
[5 5 4 4 5 5 5 5 5 4 4 4 4 5 4 5 4 4 5 4 4 5 4 4 4 4 4 4 5 4 4 4 6 5 4 4 7
 4 4 4 4]


In [44]:
# Create a copy of the DataFrame with the PCA data
df_market_data_scaled_pca_predictions = df_market_data_scaled_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_pca_predictions['clusters'] = pca_clusters

# Display sample data
df_market_data_scaled_pca_predictions.head()

,PCA1,PCA2,PCA3,clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,5
ethereum,-0.458261,0.458466,0.952877,5
tether,-0.433070,-0.168126,-0.641752,4
ripple,-0.471835,-0.222660,-0.479053,4
bitcoin-cash,-1.157800,2.041209,1.859715,5


In [45]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

pca_scatter = df_market_data_scaled_pca_predictions.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="clusters",
    hover_cols='coin_id',
    label='clusters',
    title='PCA Scatter',
    color=['Plum', 'Magenta', 'DarkViolet', 'Indigo']
).opts(yformatter="%.0f")

pca_scatter

:NdOverlay   [clusters]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [46]:
# Composite plot to contrast the Elbow curves

composite_elbow = (original_plot * pca_plot).opts(legend_position='top_right')
composite_elbow

# side_by_side_elbow = (original_plot + pca_plot)
# side_by_side_elbow

:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [47]:
# Composite plot to contrast the clusters
# composite_scatter = (original_scatter * pca_scatter)
# composite_scatter

side_by_side_scatter = (original_scatter.opts(width=450) + pca_scatter.opts(width=450))
side_by_side_scatter

:Layout
   .NdOverlay.Clusters.I  :NdOverlay   [clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.Clusters.II :NdOverlay   [clusters]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The clusters are a little tighter (less inertia) and easier to visually identify. The original data clusters have more spread and some crossover areas, making interpretation less obvious at a glance. 